In [1]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git 

  Cloning https://www.github.com/keras-team/keras-contrib.git to c:\users\husey\appdata\local\temp\pip-req-build-onp181x6


ERROR: Command errored out with exit status 128: git clone -q https://www.github.com/keras-team/keras-contrib.git 'C:\Users\husey\AppData\Local\Temp\pip-req-build-onp181x6' Check the logs for full command output.


In [2]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.layers import Activation, Conv2D, Input, Dropout, LeakyReLU, Input, Concatenate, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.initializers import RandomNormal
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization 
from random import randint

In [3]:
def discriminatorModel(imageShape):
    model = Sequential()
    model.add(Input(shape=imageShape))
    model.add(Conv2D(64, (4, 4), strides=(2,2), padding='same',input_shape=imageShape))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (4, 4), strides=(2,2), padding='same'))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (4, 4), strides=(2,2), padding='same'))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (4, 4), strides=(2,2), padding='same'))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (4, 4), strides=(1,1), padding='same'))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Conv2D(1, (4,4)))
    # We need to slow down the rate at which the descriminator learns
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'],loss_weights=[0.5])
    return model
model = discriminatorModel((256,256,3))    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      3136      
_________________________________________________________________
instance_normalization (Inst (None, 128, 128, 64)      128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       131200    
_________________________________________________________________
instance_normalization_1 (In (None, 64, 64, 128)       256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 256)       5

In [4]:
def residualBlock(LNumberofFilters, LInputLayer):
    resTensor = Conv2D(LNumberofFilters, (3,3), padding='same')(LInputLayer)
    resTensor =InstanceNormalization(axis=-1)(resTensor)
    resTensor = Activation('relu')(resTensor)
    resTensor = Conv2D(LNumberofFilters, (3,3), padding='same')(resTensor)
    resTensor =InstanceNormalization(axis=-1)(resTensor)
    resTensor = Concatenate()([resTensor, LInputLayer])
    return resTensor

In [5]:
def generatorModel(LImageShape, LResnet=9):
    LInputShape = Input(shape=LImageShape)
    genTensor = Conv2D(64, (7,7), padding = 'same',)(LInputShape)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    genTensor = Conv2D(128, (3,3), strides = (2,2), padding = 'same')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    genTensor = Conv2D(258, (3,3), strides = (2,2), padding = 'same')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    
    for _ in range(LResnet):
        genTensor = residualBlock(256, genTensor)
    
    genTensor = Conv2DTranspose(128, (3,3), strides = (2,2), padding = 'same')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Conv2DTranspose(64, (3,3), strides = (2,2), padding = 'same')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Conv2D(3, (7,7), padding='same', activation='tanh')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    
    LOutImage = Activation('tanh')(genTensor)
    model = Model(LInputShape, LOutImage)
    return model

model = generatorModel((256,256,3)) 
model.summary()  

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 256, 256, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
instance_normalization_5 (Insta (None, 256, 256, 64) 128         conv2d_6[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           instance_normalization_5[0][0]   
______________________________________________________________________________________________

In [6]:
def compositeModel(generatorModel1, discriminatorModel, generatorModel2, imageShape):
    generatorModel1.trainable = True
    discriminatorModel.trainable = False
    generatorModel2.trainable = False

    InputGenerator = Input(shape=imageShape)
    generatorModel1Out = generatorModel1(InputGenerator)
    output_d = discriminatorModel(generatorModel1Out)

    input_id = Input(shape=imageShape)
    output_id = generatorModel1(input_id)

    output_f = generatorModel2(generatorModel1Out)
    generatorModel2Out = generatorModel2(input_id)
    output_b = generatorModel1(generatorModel2Out)

    model = Model([InputGenerator, input_id], [output_d, output_id, output_f, output_b])
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer='adam')
    return model

In [7]:
imageShape = (256,256,3)
generatorModelAtoB = generatorModel(imageShape)
generatorModelBtoA = generatorModel(imageShape)
discriminatorModelA = discriminatorModel(imageShape)
discriminatorModelB = discriminatorModel(imageShape)
compositeModelAtoB = compositeModel(generatorModelAtoB, discriminatorModelB, generatorModelBtoA, imageShape)
compositeModelBtoA = compositeModel(generatorModelBtoA, discriminatorModelA, generatorModelAtoB, imageShape)

In [8]:
def generateRealSamples(dataset, LSamples, patchShape):
    ix = np.random.randint(0, dataset.shape[0], LSamples)
    X = dataset[ix]
    y = np.ones((LSamples, patchShape, patchShape, 1))
    return X, y

In [9]:
def generateFakeSamples(generatorModel, dataset, patchShape):
    X = generatorModel.predict(dataset)
    y = np.zeros((len(X), patchShape, patchShape, 1))
    return X, y

In [10]:
def update_image_pool(pool, images, max_size=50):
    selected = list()
    for image in images:
        if len(pool) < max_size:
            # stock the pool
            pool.append(image)
            selected.append(image)
        elif random() < 0.5:
            # use image, but don't add it to the pool
            selected.append(image)
        else:
            # replace an existing image and use replaced image
            ix = randint(0, len(pool))
            selected.append(pool[ix])
            pool[ix] = image
    return np.asarray(selected)

In [11]:
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(10 * 10):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    pyplot.savefig('results_baseline/generated_plot_%03d.png' % (step+1))
    pyplot.close()
    # save the generator model
    g_model.save('results_baseline/model_%03d.h5' % (step+1))

In [15]:
def train(discriminatorModelA, discriminatorModelB, generatormodelAtoB, generatormodelBtoA, 
          compositeModelAtoB, compositeModelBtoA, trainA, trainB):
    LEpochs, LBatch = 100, 1
    LPatch = discriminatorModelA.output_shape[1]
    LBatchPerEpoch = int(len(trainA) / LBatch)
    LSteps = LBatchPerEpoch * LEpochs
    LPoolA, LPoolB = list(), list()
    dA1_hist, dB1_hist, dA2_hist, dB2_hist, g1_hist, g2_hist = list(), list(), list(), list(), list(), list()
    for i in range(LSteps):
        X_realA, y_realA = generateRealSamples(trainA, LBatch, LPatch)
        X_realB, y_realB = generateRealSamples(trainB, LBatch, LPatch)
        
        X_fakeA, y_fakeA = generateFakeSamples(generatormodelBtoA, X_realB, LPatch)
        X_fakeB, y_fakeB = generateFakeSamples(generatormodelAtoB, X_realA, LPatch)
        
        X_fakeA = update_image_pool(LPoolA, X_fakeA)
        X_fakeB = update_image_pool(LPoolB, X_fakeB)
        
        gloss2, _, _, _, _  = compositeModelBtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
        
        dAloss1 = discriminatorModelA.train_on_batch(X_realA, y_realA)
        dAloss2 = discriminatorModelA.train_on_batch(X_fakeA, y_fakeA)
        
        gloss1, _, _, _, _ = compositeModelAtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
        
        dBloss1 = discriminatorModelA.train_on_batch(X_realB, y_realB)
        dBloss2 = discriminatorModelB.train_on_batch(X_fakeB, y_fakeB)
        
        #This part has something wrong with it
        #print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dAloss1, dAloss2, dBloss1, dBloss2, gloss1, gloss2))
        dA1_hist.append(dAloss1)
        dB1_hist.append(dBloss1)
        dA2_hist.append(dAloss2)
        dB2_hist.append(dBloss2)
        g1_hist.append(gloss1)
        g2_hist.append(gloss1)
        
        if (i+1) % LBatchPerEpoch == 0:
            summarize_performance(i, generatorModel, 50)
    plot_history(dA1_hist, dB1_hist, dA2_hist, dB2_hist, g1_hist, g2_hist)

In [13]:
import cv2
import os

trainA = []
dataset_path = 'images/trainA'
for filename in os.listdir(dataset_path):
    img = cv2.imread(os.path.join(dataset_path, filename))
    trainA.append(img)

trainB = []
dataset_path = 'images/trainB'
for filename in os.listdir(dataset_path):
    img = cv2.imread(os.path.join(dataset_path, filename))
    trainB.append(img)

testA = []
dataset_path = 'images/testA'
for filename in os.listdir(dataset_path):
    img = cv2.imread(os.path.join(dataset_path, filename))
    testA.append(img)

testB = []
dataset_path = 'images/testB'
for filename in os.listdir(dataset_path):
    img = cv2.imread(os.path.join(dataset_path, filename))
    testB.append(img)

trainA = np.array(trainA)
trainB = np.array(trainB)
testA = np.array(testA)
testB = np.array(testB)

In [16]:
#TO-DO: IT'S FUCKIN 6 AM AND I CAN'T GET THIS TO WORK. IT SHOULD BE EASY BUT MY STUPID COMPUTER TAKES 1:30 HOUR TO COMPILE
train(discriminatorModelA, discriminatorModelB, generatorModelAtoB, generatorModelBtoA, 
      compositeModelAtoB, compositeModelBtoA, trainA, trainB)

TypeError: 'module' object is not callable

In [17]:
#TO-DO: Test and generate image